In [1]:
from bs4 import BeautifulSoup
import requests
import bs4
import re

In [2]:
# change this to the actual url and use requests to pull the page
index = "http://www.presidency.ucsb.edu/debates.php"
response = requests.get(index)

# beautiful soup object for html parsing
soup = BeautifulSoup(response.text.encode('utf-8'))

# find all links to debate transcripts
debate_links = soup.find_all('td', attrs={'class':'doctext'})
debate_dates = soup.find_all('td', attrs={'class':'docdate'})

/Users/wallacdc/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [11]:
#debate_links

In [64]:
#transcript_urls

In [4]:
# get urls for each transcript
transcript_urls = []
n_sans_link = 0
for i in range(len(debate_links)):
    try:
        transcript_urls.append(
            (debate_links[i].select('a')[0].get('href'), 
             debate_dates[i].text.split(', ')[1])
        )
    except IndexError:
        n_sans_link += 1
        
print 'got %i urls, %i listings with no link.' % (len(transcript_urls), n_sans_link)

got 134 urls, 13 listings with no link.


In [12]:
#transcript_urls

In [8]:
#response2.text[89135:]

In [61]:
response1 = requests.get(transcript_urls[0][0])
response2 = requests.get(transcript_urls[-5][0])

In [68]:
# html starts after a fat block of javascript and XML.
# read the page after that point

# new pages have the participants names in bold
start1 = response1.text.index('<span class="displaytext">')
end1 = response1.text.index('<hr noshade="noshade" size="1">')
soup1 = BeautifulSoup(response1.text.encode('utf-8'))

# old pages do not bold the participants names
start2 = response2.text.index('<span class="displaytext">')
end2 = response2.text.index('<hr noshade="noshade" size="1">')
soup2 = BeautifulSoup(response2.text.encode('utf-8'))

In [75]:
transcript_urls[-5][0]

'http://www.presidency.ucsb.edu/ws/index.php?pid=62249'

In [13]:
#soup2

In [25]:
'Republican' in soup1.title.text

False

In [6]:
def parse_speakers_and_quotes(page, year):
    """
    Extracts names of every person involved with the debate and 
    collects all of their quotes in a list.
    
    :param page: html page
    
    :returns dict: {speaker name: [quotes]}
    """
    # fixes a BeautifulSoup problem that results in improper parsing
    page_soup = BeautifulSoup(page.text.encode('utf-8'))
    
    if 'Republican' in page_soup.title.text:
        party = 'R'
    elif 'Democratic' in page_soup.title.text:
        party = 'D'
    else:
        return 0 # only want primary debates
    
    try:
        participants = page_soup.text.lower().split('participant')[1].split('moderator')[0].strip('; ').lower()
    except IndexError:
        return 0 # reject pages that don't list participants
        
    speaker_dict = dict()
    prev_speaker = ''
    for i in page_soup.find_all('p'):

        try: # search for name of person speaking
            curr_speaker = re.findall('[A-Z]+:', i.text)[0].strip(':')
            prev_speaker = curr_speaker
            quote = re.split(':', i.text)[1]

        except IndexError: # if name not in line
            quote = i.text
            
        if prev_speaker.lower() in participants:
            #pass # only want statements from candidates
        
            if party+'_'+prev_speaker+'_'+year not in speaker_dict:
                # add speaker to speaker_dict with list of quotes as value
                speaker_dict[party+'_'+prev_speaker+'_'+year] = [quote]
            else:
                # append quote to speaker's list of quotes
                speaker_dict[party+'_'+prev_speaker+'_'+year].append(quote)
            
    return speaker_dict

In [7]:
from requests_futures.sessions import FuturesSession

start = 0
end = len(transcript_urls)
#end = 20

session = FuturesSession(max_workers=5)
futures = [session.get(url[0]) for url in transcript_urls[start:end]]

dates = [url[1] for url in transcript_urls[start:end]]
results = zip(futures, dates)

debate_transcripts = [parse_speakers_and_quotes(future.result(), date) for future, date in results]

In [8]:
import dill

In [9]:
dill.dump(debate_transcripts, open('../dills/debate_transcripts_list.dill', 'wb'))

In [7]:
#debate_transcripts

In [10]:
[i.keys() for i in debate_transcripts if type(i) == dict]

[[u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_MALLEY_2016', u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_MALLEY_2016', u'D_CLINTON_2016', u'D_SANDERS_2016'],
 [u'D_MALLEY_2015', u'D_SANDERS_2015', u'D_CLINTON_2015'],
 [u'D_SANDERS_2015',
  u'D_CLINTON_2015',
  u'D_WEBB_2015',
  u'D_MALLEY_2015',
  u'D_CHAFEE_2015',
  u'D_INTON_2015'],
 [u'R_KASICH_2015', u'R_CRUZ_2015', u'R_TRUMP_2015', u'R_RUBIO_2015'],
 [u'R_CARSON_2016',
  u'R_S_2016',
  u'R_CRUZ_2016',
  u'R_KASICH_2016',
  u'R_TRUMP_2016',
  u'R_RUBIO_2016'],
 [u'R_CARSON_2016',
  u'R_RUBIO_2016',
  u'R_CRUZ_2016',
  u'R_KASICH_2016',
  u'R_TRUMP_2016',
  u'R_BUSH_2016'],
 [u'R_CARSON_2016',
  u'R_RUBIO_2016',
  u'R_CRUZ_2016',
  u'R_CHRISTIE_2016',
  u'R_KASICH_2016',
  u'R_TRUMP_2016',
  u'R_BUSH_2016'],
 [u'R_CARSON_2016',
  u'R_RUBIO_2016',
  u'R_CRUZ_2016',
  u'R_CHRISTIE_2016',
  u'R_KASICH_2016',
  u'R